## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
from config import api_key
import requests
import time
from scipy.stats import linregress
import gmaps
from config import g_key

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2/12/2022 7:55,-23.1203,-134.9692,78.94,73,8,17.52,light rain
1,1,Mahebourg,MU,2/12/2022 7:55,-20.4081,57.7000,84.25,78,40,13.80,scattered clouds
2,2,Jamestown,US,2/12/2022 7:55,42.0970,-79.2353,38.08,66,100,11.01,overcast clouds
3,3,Ushuaia,AR,2/12/2022 7:52,-54.8000,-68.3000,51.46,62,40,51.79,scattered clouds
4,4,Puerto Ayora,EC,2/12/2022 7:55,-0.7393,-90.3518,71.55,97,100,1.01,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2/12/2022 7:55,-23.1203,-134.9692,78.94,73,8,17.52,light rain
1,1,Mahebourg,MU,2/12/2022 7:55,-20.4081,57.7000,84.25,78,40,13.80,scattered clouds
4,4,Puerto Ayora,EC,2/12/2022 7:55,-0.7393,-90.3518,71.55,97,100,1.01,overcast clouds
10,10,Liuli,TZ,2/12/2022 7:55,-11.0833,34.6333,75.78,82,100,3.94,light rain
16,16,Avarua,CK,2/12/2022 7:55,-21.2078,-159.7750,75.25,94,46,1.14,scattered clouds
18,18,Nelson Bay,AU,2/12/2022 7:55,-32.7167,152.1500,74.28,65,63,1.99,broken clouds
20,20,Cape Town,ZA,2/12/2022 7:55,-33.9258,18.4232,79.88,56,0,8.05,clear sky
22,22,Port Hedland,AU,2/12/2022 7:55,-20.3167,118.5667,89.65,70,9,9.22,light rain
24,24,Busselton,AU,2/12/2022 7:55,-33.6500,115.3333,75.09,45,6,12.82,clear sky
27,27,Kulhudhuffushi,MV,2/12/2022 7:55,6.6221,73.0700,81.79,69,48,1.72,moderate rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.94,light rain,-23.1203,-134.9692,
1,Mahebourg,MU,84.25,scattered clouds,-20.4081,57.7000,
4,Puerto Ayora,EC,71.55,overcast clouds,-0.7393,-90.3518,
10,Liuli,TZ,75.78,light rain,-11.0833,34.6333,
16,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
18,Nelson Bay,AU,74.28,broken clouds,-32.7167,152.1500,
20,Cape Town,ZA,79.88,clear sky,-33.9258,18.4232,
22,Port Hedland,AU,89.65,light rain,-20.3167,118.5667,
24,Busselton,AU,75.09,clear sky,-33.6500,115.3333,
27,Kulhudhuffushi,MV,81.79,moderate rain,6.6221,73.0700,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.94,light rain,-23.1203,-134.9692,People ThankYou
1,Mahebourg,MU,84.25,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
4,Puerto Ayora,EC,71.55,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Liuli,TZ,75.78,light rain,-11.0833,34.6333,"Puulu Beach,Puulu Village"
16,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,Paradise Inn


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
print(fig)

Figure(layout=FigureLayout(height='420px'))
